# EIS DEM Slope Maps

In [1]:
import pathlib

import asdf
import astropy.units as u
import astropy.table

from synthesizAR.analysis import make_slope_map
from synthesizAR.instruments.util import read_cube_from_dataset

/Users/wtbarnes/mambaforge/envs/agu-2024-poster/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = pathlib.Path('../../../talks/loops-workshop-2024-talk/data/')
figure_dir = pathlib.Path('../src/figures/')

In [3]:
slope_fitting_params = {
    'temperature_bounds': (0.9*u.MK,3*u.MK),
    'em_threshold': 1e24*u.cm**(-5),
    'rsquared_tolerance': 0.5,
    'max_upper_bound': 8*u.MK,
    'mask_negative': True,
}

In [4]:
ar_table = astropy.table.QTable.read(data_dir / 'tables' / 'mason_ars_region_1.asdf')

In [5]:
for i,row in enumerate(ar_table):
    em_dir = data_dir / f'noaa_{row["NOAA AR"]}' / 'diagnostics' / 'dem' 
    dem_res = read_cube_from_dataset(
        em_dir / f'em_noaa{row["NOAA AR"]}_eis.nc',
        'temperature',
        'phys.temperature',
    )
    slope_map = make_slope_map(dem_res, **slope_fitting_params)
    with asdf.AsdfFile({'slope_map': slope_map}) as asdf_file:
        asdf_file.write_to(em_dir / f'emslope_noaa{row["NOAA AR"]}_eis.asdf')